In [55]:
import requests
import csv
import time
from bs4 import BeautifulSoup
from collections import Counter
import praw
import pandas as pd
import datetime as dt #only if you want to analyze the date created feature
import json 
from types import SimpleNamespace
import sys
from nltk.tokenize import TreebankWordTokenizer

Fetch the dictionary dataset, convert it to a list of JSON objects of this form:
 
 {
 
    'user': 'username',
    'posts': [{
        'sub': 'subreddit name here',
        'title': 'title of post',
        'body': 'body of post',
        'url': 'url of post / photo',
        'num_u': 'number of upvotes',
        'num_c': 'number of comments',
        'time': 'local time of posting'
    }]
 
 }

In [65]:
with open("dataset/small-trial.json", "r") as read_file:
    data = json.load(read_file)
    
users_list = []

for users_posts in data.items():
    users_list.append({'user': users_posts[0], 'posts': users_posts[1]})

In [92]:
def get_tokens(string, to_lower): # TODO : Change to BERT / explore more as we see fit
    ''' String ---> Tokenize ---> Lowercase (if set to true) '''
    return TreebankWordTokenizer().tokenize(string) if to_lower is False else \
        [token.lower() for token in TreebankWordTokenizer().tokenize(string)]

def tokenize_posts(user, to_lower):
    '''Iterates through each post ---> Tokenizes title and body'''
    for post in user['posts']:
        tokenized_title = get_tokens(post['title'], to_lower)
        tokenized_body = get_tokens(post['body'], to_lower)
        post['title_tok'] = tokenized_title
        post['body_tok'] = tokenized_body
    return user

def tokenize_all_users():
    count = 0
    ''' Iterates through each user ---> Extracts posts ---> Tokenizes them'''
    for user in users_list[0:4]:
        if(count%50==0): 
            print("Tokenizing user ", count)
            count += 1
        tokenized_user_posts = tokenize_posts(user, True)
        for post in tokenized_user_posts['posts']:
            print(post, "\n\n")

In [90]:
tokenize_all_users()

Tokenizing user  0
{'sub': 'Psychic', 'title': 'Does anyone here know how to interpret dreams?', 'body': 'I had a dream last night that I’m trying to figure out the reason behind. \nAlso the energy around me has been making feel a certain way, I think it may be the moon.', 'num_u': 1, 'url': 'https://www.reddit.com/r/Psychic/comments/gnl0ok/does_anyone_here_know_how_to_interpret_dreams/', 'num_c': 5, 'time': 1590044076.0, 'title_tok': ['does', 'anyone', 'here', 'know', 'how', 'to', 'interpret', 'dreams', '?'], 'body_tok': ['i', 'had', 'a', 'dream', 'last', 'night', 'that', 'i', '’', 'm', 'trying', 'to', 'figure', 'out', 'the', 'reason', 'behind.', 'also', 'the', 'energy', 'around', 'me', 'has', 'been', 'making', 'feel', 'a', 'certain', 'way', ',', 'i', 'think', 'it', 'may', 'be', 'the', 'moon', '.']} 


{'sub': 'MakeNewFriendsHere', 'title': '19F having a lot of anxiety, could really use someone to talk to please', 'body': 'Hello all, I’m 19 and I’m having a lot of anxiety right now th

In [ ]:
def write_to_csv():
    with open("output.csv","w",newline="") as f:  # python 2: open("output.csv","wb")
        title = "time,SOURCE,PLACE,TEMP,LIGHT,HUMIDITY".split(",") # quick hack
        cw = csv.DictWriter(f,title,delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        cw.writeheader()
        cw.writerows(data)
        
  

In [114]:
import pandas
pd = pandas.read_json("dataset/reddit-posts-mental.json")
pd['label'] = 1
pd.to_csv('dataset/reddit-posts-mental.csv')

pd = pandas.read_json("dataset/reddit-posts-anxiety.json")
pd['label'] = 1
pd.to_csv('dataset/reddit-posts-anxiety.csv')

pd = pandas.read_json("dataset/reddit-posts-unrelated.json")
pd['label'] = 1
pd.to_csv('dataset/reddit-posts-unrelated.csv')

pd = pandas.read_json("dataset/reddit-control.json")
pd['label'] = 0 
pd.to_csv('dataset/reddit-control.csv')

In [116]:
print(pd)

             sub                                              title  \
0      AskReddit  People who haven't pooped in 2019 yet, why are...   
1      AskReddit  How would you feel about a feature where if so...   
2      AskReddit  Would you watch a show where a billionaire CEO...   
3      AskReddit           Stan Lee has passed away at 95 years old   
4      AskReddit  Reddit, how would you feel about a law that ba...   
...          ...                                                ...   
18313       rant  Stop acting like adoption is second-tier paren...   
18314       rant  Dear the person who followed me just to downvo...   
18315       rant                      Reddit is a toxic circle-jerk   
18316       rant                  South Korea is destroying my life   
18317       rant  How hard is it to be on time for once in your ...   

                                                    body   num_u  \
0                                                         221854   
1          

In [97]:
# from fast_bert.data_cls import BertDataBunch

# databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
#                           tokenizer='bert-base-uncased',
#                           train_file='train.csv',
#                           val_file='val.csv',
#                           label_file='labels.csv',
#                           text_col='text',
#                           label_col='label',
#                           batch_size_per_gpu=16,
#                           max_seq_length=512,
#                           multi_gpu=True,
#                           multi_label=False,
#                           model_type='bert')